In [1]:
import numpy as np
from xgbsurv.models.eh_final import eh_likelihood, eh_gradient
from xgbsurv.models.eh_ah_final import ah_likelihood
import sys
sys.path.append('/Users/JUSC/Documents/xgbsurv_benchmarking/deep_learning/')
from loss_functions_pytorch import eh_likelihood_torch, eh_likelihood_torch_2, aft_likelihood_torch, ah_likelihood_torch
import torch

## 1. Compare Pytorch loss with numpy loss

- let's keep bandwidth fixed for that
- random data

linear_predictor = np.array([[0.67254923, 0.03356795],
       [0.86077982, 0.65922692],
       [0.43557393, 0.75447972],
       [0.94059047, 0.30572004],
       [0.8446509 , 0.07916267],
       [0.23657039, 0.44693716],
       [0.74629685, 0.32637245],
       [0.99700768, 0.10225456],
       [0.28182768, 0.05405025],
       [0.44495038, 0.08454563]])

y = np.array([1, -3, -3, -4, -7,  8,  9,  -11,  13,  16])


In [2]:
# create mock data
linear_predictor = np.array([[0.67254923, 0.03356795],
       [0.86077982, 0.65922692],
       [0.43557393, 0.75447972],
       [0.94059047, 0.30572004],
       [0.8446509 , 0.07916267],
       [0.23657039, 0.44693716],
       [0.74629685, 0.32637245],
       [0.99700768, 0.10225456],
       [0.28182768, 0.05405025],
       [0.44495038, 0.08454563]])
linear_predictor_torch = torch.from_numpy(linear_predictor)

y = np.array([[1, -3, -3, -4, -7,  8,  9,  -11,  13,  16],
              [1, -3, -3, -4, -7,  8,  9,  -11,  13,  16]]).reshape(10,2)
y_torch = torch.from_numpy(y[:,0])

In [3]:
linear_predictor = np.repeat(linear_predictor[:,0],2).reshape(10,2)

In [4]:
l = eh_likelihood(y, linear_predictor)
l

1.597173328739984

In [5]:
#l1= eh_likelihood_torch(y_torch, linear_predictor_torch)
#l1.numpy()

In [6]:
#np.isclose(l,l1)

In [7]:
#eh_likelihood_torch_2(y_torch, linear_predictor_torch).numpy()

## 2. Compare Pytorch gradient with gradient function

In [12]:
# create mock data
linear_predictor_torch =  torch.tensor([[0.67254923, 0.03356795],
       [0.86077982, 0.65922692],
       [0.43557393, 0.75447972],
       [0.94059047, 0.30572004],
       [0.8446509 , 0.07916267],
       [0.23657039, 0.44693716],
       [0.74629685, 0.32637245],
       [0.99700768, 0.10225456],
       [0.28182768, 0.05405025],
       [0.44495038, 0.08454563]], requires_grad=True)


linear_predictor_torch =torch.tensor(np.repeat(linear_predictor[:,0],2).reshape(10,2), requires_grad=True)
y_torch = torch.tensor([1., -3., -3., -4., -7.,  8.,  9.,  -11.,  13.,  16.])


In [9]:
linear_predictor_torch

tensor([[0.6725, 0.6725],
        [0.8608, 0.8608],
        [0.4356, 0.4356],
        [0.9406, 0.9406],
        [0.8447, 0.8447],
        [0.2366, 0.2366],
        [0.7463, 0.7463],
        [0.9970, 0.9970],
        [0.2818, 0.2818],
        [0.4450, 0.4450]], dtype=torch.float64, requires_grad=True)

In [10]:
l1= eh_likelihood_torch_2(linear_predictor_torch, y_torch)
#l1.retain_grad()
l1.backward()
#print(linear_predictor_torch.grad)
print(linear_predictor_torch.grad.numpy())

[[ 0.02967589 -0.09394635]
 [ 0.00416697  0.02602212]
 [ 0.00214852  0.01591823]
 [ 0.00302787  0.03937925]
 [-0.00916972  0.06180726]
 [-0.0424455  -0.06107999]
 [ 0.01515106 -0.03006867]
 [-0.04652952  0.09290624]
 [ 0.00476166 -0.03522046]
 [ 0.03921276 -0.01571763]]


In [11]:
a1 = ah_likelihood_torch(linear_predictor_torch, y_torch)
a1.backward()
#print(linear_predictor_torch.grad)
print(linear_predictor_torch.grad.numpy())

RuntimeError: The size of tensor a (10) must match the size of tensor b (2) at non-singleton dimension 1

In [ ]:

y = np.array([[1., -3., -3., -4., -7.,  8.,  9.,  -11.,  13.,  16.],
              [1., -3., -3., -4., -7.,  8.,  9.,  -11.,  13.,  16.]]).reshape(10,2)
eh_gradient(y, np.array(linear_predictor)) #.reshape(-1,2)

array([ 0.15160797, -0.00500409, -0.003691  ,  0.18905714,  0.26316123,
        0.05587306, -0.00582405, -0.01833184,  0.04190684,  0.15520722])

In [ ]:
np.array(linear_predictor)

array([[0.67254923, 0.67254923],
       [0.86077982, 0.86077982],
       [0.43557393, 0.43557393],
       [0.94059047, 0.94059047],
       [0.8446509 , 0.8446509 ],
       [0.23657039, 0.23657039],
       [0.74629685, 0.74629685],
       [0.99700768, 0.99700768],
       [0.28182768, 0.28182768],
       [0.44495038, 0.44495038]])

In [ ]:
y_torch.repeat(2)

tensor([  1.,  -3.,  -3.,  -4.,  -7.,   8.,   9., -11.,  13.,  16.,   1.,  -3.,
         -3.,  -4.,  -7.,   8.,   9., -11.,  13.,  16.])

In [ ]:
y_torch.tile(2)

tensor([  1.,  -3.,  -3.,  -4.,  -7.,   8.,   9., -11.,  13.,  16.,   1.,  -3.,
         -3.,  -4.,  -7.,   8.,   9., -11.,  13.,  16.])

In [ ]:
import math
n_samples: int = y.shape[0]
bandwidth = 1.30 * math.pow(n_samples, -0.2)
linear_predictor_torch = linear_predictor_torch[:,1]
l1= aft_likelihood_torch(y_torch, linear_predictor_torch[:,1], sample_weight=1.0, bandwidth = bandwidth )
l1.retain_grad()
l1.backward()
print(linear_predictor_torch.grad)

IndexError: too many indices for tensor of dimension 1